In [7]:
import numpy as np

In [8]:
def fft(x):
    return np.fft.fft(x)
def ifft(X):
    return np.fft.ifft(X)

In [9]:
def overlap_add(x, h, N):
    L = len(x)
    M = len(h)
    h_padded = np.concatenate((h, np.zeros(N - M)))
    H = fft(h_padded)
    output = np.zeros(L + M - 1)
    for i in range(0, L, N - M + 1):
        chunk = x[i:i + N - M + 1]
        chunk = np.concatenate((chunk, np.zeros(M - 1)))
        X = fft(chunk)
        Y = X * H
        y = ifft(Y)
        output[i:i + N - M + 1 + M - 1] += np.real(y)
    return output[:L + M - 1]

In [10]:
def overlap_save(x, h, N):
    L = len(x)
    M = len(h)
    # Ensure N is large enough to cover x + h - 1
    if N < L + M - 1:
        N = L + M - 1
    # Zero-pad filter h to length N - 1
    h_padded = np.concatenate((h, np.zeros(N - M)))
    H = fft(h_padded)
    x = np.concatenate((np.zeros(M - 1), x))
    output = np.zeros(L)
    for i in range(0, L, N - M + 1):
        chunk = x[i:i + N]
        chunk_padded = np.concatenate((chunk, np.zeros(N - len(chunk))))
        X = fft(chunk_padded)
        Y = X * H
        y = ifft(Y)
        output[i:i + N - M + 1] += np.real(y[M - 1:M + N - M])
    return output[:L + M - 1]


### Input sequence

In [11]:
x_oam = np.array([1, 2, 3, 4, 5, 6, 1, 1, 1, 1, 1, 1, 0, 1, 2, 3, 4, 5])
h = np.array([1, 1, 1])
N = 8 # FFT size
L = 6 # Length of the input sequence
M = len(h)

### OAM

In [12]:
y_oam = overlap_add(x_oam, h, N)
print("Overlap Add Method Output (y[n]):")
print(y_oam)

Overlap Add Method Output (y[n]):
[ 1.  3.  6.  9. 12. 15. 12.  8.  3.  3.  3.  3.  2.  2.  3.  6.  9. 12.
  9.  5.]


### OSM

In [13]:
y_osm = overlap_save(x_oam, h, N)
print("Overlap Save Method Output (y[n]):")
print(y_osm)

Overlap Save Method Output (y[n]):
[ 1.  3.  6.  9. 12. 15. 12.  8.  3.  3.  3.  3.  2.  2.  3.  6.  9. 12.]
